In [1]:


!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8b5f2523dd285416b086f679927fa088b4bab4c58c81613072c6df9dabf51f9e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
spark=SparkSession.builder.appName('Indian Food').getOrCreate()

In [3]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
df=spark.read.csv('indian_food.csv',header=True,inferSchema=True)

In [7]:
df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [8]:


#Find out how many unique dishes are present.
df.select(col('name')).distinct().count()

255

In [9]:
#Which state has more dishes?
df1=df.select(col('state')).groupBy('state').count()
df1.select('state','count').filter(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()

+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



In [10]:
#How many dishes from state Karnataka?
df.where(col('state')=='Karnataka').count()

6

In [11]:
df.select(col('region')).distinct().show()

+----------+
|    region|
+----------+
|        -1|
|     South|
|   Central|
|      East|
|      West|
|North East|
|     North|
|      NULL|
+----------+



In [12]:
df.select(col('region')).groupBy('region').count().show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [13]:
df.select(col('flavor_profile'),col('course')).distinct().show()

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



In [14]:
df1=df.select('state').groupBy('state').count()
df1.select('state','count').where(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()

+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



In [17]:
total=df.count()
df.select('region').groupBy('region').count().withColumn('percentage',col('count')/ total*100).show()

+----------+-----+-------------------+
|    region|count|         percentage|
+----------+-----+-------------------+
|        -1|   13|  5.098039215686274|
|      NULL|    1|0.39215686274509803|
|     South|   59| 23.137254901960784|
|   Central|    3| 1.1764705882352942|
|      East|   31| 12.156862745098039|
|      West|   74|  29.01960784313726|
|North East|   25|  9.803921568627452|
|     North|   49| 19.215686274509807|
+----------+-----+-------------------+



In [18]:
#List the states which has more dishes from each region.
df1=df.select('state','region').groupBy('state','region').count()
df1.select('state','region','count').filter(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()

+-------+------+-----+
|  state|region|count|
+-------+------+-----+
|Gujarat|  West|   35|
+-------+------+-----+

